This notebook applies morphological smoothing and rule-based reclassification using external layers including DE Africas crop mask product.

### load packages

In [ ]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk,area_closing
from skimage.filters.rank import modal
from skimage.segmentation import expand_labels
from odc.algo import xr_reproject
import matplotlib.pyplot as plt

### input files paths and parameters

In [ ]:
output_crs='epsg:32735' # WGS84/UTM Zone 35S
dict_map={'Forest':1,'Grassland':5,'Shrubland':7,'Perennial Cropland':9,'Annual Cropland':10,
          'Wetland':11,'Water Body':12,'Urban Settlement':13,'Bare Soil':14}
# file paths and attributes
rwanda_tiles_shp='Data/Rwanda_tiles.shp'
river_network_shp='Data/hotosm_rwa_waterways_lines.shp' # OSM river network data
road_network_shp='Data/hotosm_rwa_roads_lines_filtered.shp' # OSM road network data
google_building_raster='Data/GoogleBuildingLayer_Rwanda_reprojected_rasterised.tif' # google bulding layer:
hand_raster='Data/hand_Rwanda.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
wsf2019_raster='Data/WSF2019_v1_Rwanda_clipped.tif' # 2019 WSF raster
parks_json='Data/National Parks.geojson'
wetlands_json='Data/Wetland.geojson'

classification2021_raster='Results/Land_cover_prediction_Rwanda_2021_tiles_mosaic.tif' # land cover map of 2021

### load layers

In [ ]:
rwanda_tiles=gpd.read_file(rwanda_tiles_shp).to_crs(output_crs) # get bounding boxes of tiles covering rwanda
tile_bboxes=rwanda_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021

# load external layers
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
road_network_mask=xr_rasterize(gdf=road_network,da=landcover2021.squeeze(),
                               transform=landcover2021.geobox.transform,crs=output_crs) # # rasterise buffered OSM road network layer

river_network=gpd.read_file(river_network_shp).to_crs(output_crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute
river_network_mask=xr_rasterize(gdf=river_network,da=landcover2021.squeeze(),
                                transform=landcover2021.geobox.transform,crs=output_crs) # rasterise OSM river network layer

hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze() # import hand layer

google_buildings_mask=xr.open_dataset(google_building_raster,engine="rasterio").squeeze() # import google buildings layer

wsf2019=xr.open_dataset(wsf2019_raster,engine="rasterio").astype(np.int32).squeeze() # import WSF2019 layers

# load national parks and wetlands layers
parks=gpd.read_file(parks_json).to_crs(output_crs)
parks=parks.loc[parks['type'].isin(['National Park','Volcanoes National Park'])] # select by type
parks_mask=xr_rasterize(gdf=parks,da=landcover2021.squeeze(),
                        transform=landcover2021.geobox.transform,crs=output_crs)

wetlands=gpd.read_file(wetlands_json).to_crs(output_crs)
wetlands_mask=xr_rasterize(gdf=wetlands,da=landcover2021.squeeze(),
                        transform=landcover2021.geobox.transform,crs=output_crs)

### loop through tiles for reclassification and export as geotiffs

In [ ]:
# for i in range(len(tile_bboxes)):
for i in range(0,2):# testing
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    
    # load DE Africa crop mask 2019
    dc = datacube.Datacube(app='cropland_extent')
    query = {
        'time': ('2019'),
        'x': (x_min,x_max),
        'y': (y_min,y_max),
        'resolution':(-10, 10),
        'crs':output_crs,
        'output_crs': output_crs,
    }
    # now load the crop-mask using the query
    cm = dc.load(product='crop_mask',
                 **query).squeeze()
    ds_geobox=cm.geobox
    np_crop_mask=cm['mask'].to_numpy()
        
    # clip land cover map 2021 to tile boundary
    landcover2021_tile=xr_reproject(landcover2021, ds_geobox, resampling="nearest") # clip to tile boundary
    np_landcover2021=landcover2021_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post=np_landcover2021.copy() # initialise post-processed numpy array
    
    # mode filtering for a smoother classification map
    np_landcover2021_post=modal(np_landcover2021_post,footprint=disk(1),mask=np_landcover2021_post!=0)
    
    # assign cropland pixels outside crop mask as Shrubland (Perenial Cropland) or Grassland (Annual Cropland)
    np_landcover2021_post[(np_landcover2021_post==dict_map['Perennial Cropland'])&(np_crop_mask!=1)]=dict_map['Shrubland']
    np_landcover2021_post[(np_landcover2021_post==dict_map['Annual Cropland'])&(np_crop_mask!=1)]=dict_map['Grassland']
    
    # merging Perennial Cropland and Annual cropland as Cropland
    np_landcover2021_post[np_landcover2021_post==dict_map['Perennial Cropland']]=dict_map['Annual Cropland']
    
    # assign cropland pixels within national parks as Grassland
    parks_mask_tile=xr_reproject(parks_mask, ds_geobox, resampling="nearest")
    np_parks_mask=parks_mask_tile.squeeze().to_numpy()
    np_landcover2021_post[(np_landcover2021_post==dict_map['Annual Cropland'])&(np_parks_mask==1)]=dict_map['Grassland']
    
    # assign all classes within crop mask except Grassland as Cropland
    np_landcover2021_post[(np_landcover2021_post!=dict_map['Grassland'])&(np_crop_mask==1)]=dict_map['Annual Cropland']
    
    # assign Urban Settlement pixels within national parks as surrounding classes
    temp=np_landcover2021_post.copy()
    temp[(np_landcover2021_post==dict_map['Urban Settlement'])&(np_parks_mask==1)]=0 # assign the regions as background
    temp_closed=expand_labels(temp,distance=10000) # expand surrounding classes
    mask=(temp!=temp_closed) # identify filled/changed areas
    np_landcover2021_post[mask]=temp_closed[mask] # copy the filled/changed pixels
    
    # assign wetlands pixels outside Wetlands polygons as shrubland
    wetlands_mask_tile=xr_reproject(wetlands_mask, ds_geobox, resampling="nearest")
    np_wetlands_mask=wetlands_mask_tile.squeeze().to_numpy()
    np_landcover2021_post[(np_landcover2021_post==dict_map['Wetland'])&(np_wetlands_mask!=1)]=dict_map['Shrubland']
    
#     # assign wetlands outside Wetlands polygons as neighbouring class
#     wetlands_mask_tile=xr_reproject(wetlands_mask, ds_geobox, resampling="nearest")
#     np_wetlands_mask=wetlands_mask_tile.squeeze().to_numpy()
#     temp=np_landcover2021_post.copy()
#     temp[(np_landcover2021_post==dict_map['Wetland'])&(np_wetlands_mask!=1)]=0
#     temp_closed=expand_labels(temp,distance=10000)
#     mask=(temp!=temp_closed)
#     np_landcover2021_post[mask]=temp_closed[mask]
    
    #     # reclassify wetlands around (within 50m of) built-up areas as Forest
#     urban_buffered=binary_dilation(np_landcover2021_post==dict_map['Urban Settlement'],footprint=disk(5)) # dilating built-up regions
#     np_landcover2021_post[(urban_buffered==1)&(np_landcover2021_post==dict_map['Wetland'])]=dict_map['Forest'] # apply rule
    
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    # assign water pixels outside these areas as surrounding class
    hand_tile=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand_tile.to_array().squeeze().to_numpy()
    river_network_mask_tile=xr_reproject(river_network_mask, ds_geobox, resampling="nearest")
    np_river_network_mask=river_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    temp=np_landcover2021_post.copy()
    temp[(np_landcover2021_post==dict_map['Water Body'])&(np_hand>45)&(np_river_network_mask!=1)]=0
    temp_closed=expand_labels(temp,distance=10000)
    mask=(temp!=temp_closed)
    np_landcover2021_post[mask]=temp_closed[mask]
#     np_landcover2021_post[((np_landcover2021_post==dict_map['Water Body'])&(np_hand<=45))|(np_river_network_mask==1)]=dict_map['Water Body'] # apply rules

    # assign pixels overlapping OSM river network as Water Body
    np_landcover2021_post[np_river_network_mask==1]=dict_map['Water Body']
    
    # assign pixels overlapping google building polygons or WSF 2019 as built-up
    google_buildings_mask_tile=xr_reproject(google_buildings_mask, ds_geobox, resampling="nearest")
    np_google_buildings_mask=google_buildings_mask_tile.to_array().squeeze().to_numpy() # data array to numpy array
    wsf2019_tile=xr_reproject(wsf2019, ds_geobox, resampling="nearest") # load and clip WSF layers
    np_wsf2019=wsf2019_tile.to_array().squeeze().to_numpy()
    np_landcover2021_post[(np_google_buildings_mask==1)|(np_wsf2019==255)]=dict_map['Urban Settlement'] # apply rules
    
    # assign pixels overlapping buffered OSM road network as built-up class
    road_network_mask_tile=xr_reproject(road_network_mask, ds_geobox, resampling="nearest")
    np_road_network_mask=road_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post[np_road_network_mask==1]=dict_map['Urban Settlement'] # burn in buffered OSM road network polygons
    
    # convert result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_postprocessed_Rwanda_2021_tile_'+str(i)+'.tif', overwrite=True)

### mosaic all post-processed tiles

In [ ]:
! gdal_merge.py -o Results/Land_cover_prediction_postprocessed_Rwanda_2021_tiles_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover_prediction_postprocessed_Rwanda_2021_tile_*.tif